<a href="https://colab.research.google.com/github/achanhon/coursdeeplearningcolab/blob/master/bow_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP Sac de mot

L'objectif de ce tp est de coder une approche sac de mots : c'est ce qu'on faisait de mieux **avant** le deep learning.


On va réutiliser MNIST que vous connaissez désormais.


## Création d'un dictionnaire

On va extraire un ensemble DENSE de patch dans des images MNIST et appliquer K moyenne dessus, cela nous donnera un dictionnaire de patches !

D'abord récupérons MNIST (avec torchvision -- juste par faciliter mais ce n'est pas pertinent).

In [ ]:
import torch
import torchvision
transform_mnist = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32,32)),
    torchvision.transforms.ToTensor()
])
mnisttrain = torchvision.datasets.MNIST("./mnist",train=True, transform=transform_mnist, download=True)
trainloader = torch.utils.data.DataLoader(mnisttrain, batch_size=64, shuffle=True, num_workers=2)
mnisttest = torchvision.datasets.MNIST("./mnist",train=False, transform=transform_mnist, download=True)
testloader = torch.utils.data.DataLoader(mnisttest, batch_size=64, shuffle=True, num_workers=2)

import matplotlib.pyplot as plt
import numpy as np
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

sample = next(iter(trainloader))[0]
show(torchvision.utils.make_grid(sample))
print(sample.shape)  ## 64 c'est le batch
                        ## 1 c'est du gris -- sinon ce serait 3 pour du RGB
                        ## 32x32 c'est pour la taille de l'image (petite ici)

**Q1** (moyen) : écrire une fonction qui extrait des patch 8x8 dans une image, les transforme en vecteur (flatten) et les mets dans une liste donnée en argument

In [ ]:
def extraitpatch2D(allpatch,inputs):
  print("TODO")

def extraitpatchinbatch(allpatch,inputs):
  for i in range(inputs.shape[0]):
      extraitpatch2D(allpatch,inputs[i][0])

**ATTENTION :** en fait c'est vraiment consommateur de ressource d'extraire autant de patches...

On vous fourniera une version qui tire au hasard des patchs à extraire (et qui repasse en numpy).

**Q2 (facile)** : visualiser 64 patchs tirés au hasard et/ou les patchs extraits d'une image unique. Chacun apporte une information "faible" mais dans leur ensemble, on espère qu'ils encodent l'image.

In [ ]:
print("TODO")

**Q3 (facile)** Parcourer le dataset de train pour extraire l'ensemble (avec prunning) des patchs de l'ensemble des images (normalement on utilise train **ET** test mais bon)

In [ ]:
allpatch = []
print("TODO")

**Q4 (moyen)** : Appliquer sklearn.cluster.KMeans pour créer un dictionnaire de 2048 clusters

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html


In [ ]:
print("TODO")

## Encodage

L'objectif maintenant est d'utiliser le dictionnaire pour créer une fonction qui prend une image en entrée et qui produit un histogramme H avec h_i qui correspond au nombre de fois qu'on a vu un patch qui se rattache au groupe "i" du dictionnaire !

cette fois on extraira tous les patchs -- vu que de toute façon on va juste en conserver leur index


In [ ]:
def extractallpatch(image):
  out = []
  for row in range(image.shape[0]-8):
    for col in range(image.shape[1]-8):
        out.append(image[row:row+8,col:col+8].copy().flatten())
  return out


**Q5 (moyen)** : regarder l'exemple donnée dans la doc de scikit -- il vous permet de voir comment avoir le label associé à un patch

écrire une fonction qui prend 1 image et produit l'histogramme associé

In [ ]:
def encodeimage(image):
  print("TODO")

## Classification

**Q6 (dur voire bonus)** : parcourir l'ensemble du dataset de train, produire et stocker l'ensemble des histogrammes, puis apprendre un MLP (2 couches) ou arbre ou  sur ces histogrammes !

maintenant on definit le réseau

In [ ]:
class MonReseau(nn.Module):
    def __init__(self):
        super(MonReseau, self).__init__()
        
        self.linear1 = nn.Linear(1024, 1024)
        self.linear2 = nn.Linear(1024, 2048)
        self.linear3 = nn.Linear(2048, 4096)

        self.final = nn.Linear(4096,10)
      
    def forward(self, x):
        x = x.view(-1,1024)  # l'image 1 x 32 x 32 devient un vecteur 1024
        x = F.leaky_relu(self.linear1(x))
        x = F.leaky_relu(self.linear2(x))
        x = F.leaky_relu(self.linear3(x))
        
        x = self.final(x)
        return x

monreseau = MonReseau()
monreseau = monreseau.cuda()

ensuite on définit les paramètres de l'apprentissage : 
- le solver
- la loss
- le nombre d'itérations ...

In [ ]:
optimizer = optim.Adam(monreseau.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()
nbepoch = 5

l'apprentissage à proprement parler :
- on itere sur les données
- on les fait rentrer dans le réseau
- on compare la sortie courante à la sortie voulue
- on calcule le gradient 
- on actualise les poids pour que la sortie courante soit plus proche que la sortie voulue

**ET ON PASSE SUR CUDA CAR SINON C'EST TROP LONG**

In [ ]:
import random
for epoch in range(nbepoch):
  monreseau.train()
  print("epoch", epoch)
  for inputs, targets in trainloader:   ## on itere sur les données 
    inputs, targets = inputs.cuda(),targets.cuda()

    mespredictions = monreseau(inputs)  ## on les fait rentrer dans le réseau
    loss = criterion(mespredictions,targets)  ## on compare la sortie courante à la sortie voulue

    loss.backward() ## le gradient -- la magie par rapport à comment c'était long en court :-)
    optimizer.step() ## on actualise les poids pour que la sortie courante soit plus proche que la sortie voulue

    if random.randint(0,90)==0:
      print("\tloss=",loss) ## on affiche pour valider que ça diverge pas


Maintenant, on calcule la performance obtenue **en test**
à travers la matrice de confusion
Mij c'est le nombre de fois qu'une image de la classe i a été classé comme j

In [ ]:
from sklearn.metrics import confusion_matrix

monreseau.eval()
cm = np.zeros((10,10),dtype=int)  ###### ATTENTION DE BIEN ME REMETTRE A ZERO
with torch.no_grad():  ### ici pas besoin de calculer les gradients
    for inputs, targets in testloader:
        inputs = inputs.cuda()
        outputs = monreseau(inputs)
        _,pred = outputs.max(1)
        cm += confusion_matrix(pred.cpu().numpy(),targets.cpu().numpy(),list(range(10)))
print(cm)

**Q1 (facile)** : calculer le taux de bonne classification (donc la diagonale sur le total)

In [ ]:
print("TODO")

### CNN

**Q2 (dur)** : maintenant il faut changer le réseau pour mettre des convolutions et donc traiter la spécificité des images plutôt que de considérer ça comme un vecteur.

Dupliquer le code de "MonReseau" et implémenter lenet au lieu d'un simple mlp

regarder la doc : https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

et regarder sur internet pour lenet vous aurez plein d'info !!

In [ ]:
print("TODO")

Lancez maintenant un train/test

vous pouvez réutiliser quasiment tel quel la boucle "d'apprentissage" -- elle ne dépend PAS du réseau (les paramètres pourraient peut être changer mais dans les grosses lignes c'est la même boucle)

In [ ]:
print("TODO")

# CNN VS MLP

Maintenant on va s'intéresser à la différence entre un mlp et un CNN


on fait regarder ce que donne nos codes -- si on permute les pixels
ce sera évidemment une permutation **FIXE** (ce sera la même pour toutes les images train et test) qu'on peut générer calculant une permutation des lignes et une des colonnes

In [ ]:
def computerandompermutation(n):
	out = list(range(n))
	random.shuffle(out)
	return out
permrow,permcol = computerandompermutation(32),computerandompermutation(32)
print(permrow)



**Q3 (moyenne) :** Coder une fonction qui prends en argument une permutation et une image et qui permute ses pixels.


*n'oubliez est pour permuter la valeur de x et y dans un tableau T il faut faire*
- tmp = T[y]
- T[y] = T[x]
- T[x] = tmp

(sinon on écrase une des 2 valeurs)

commencer par permuter une image 32x32 et créer une fonction qui l'applique 64 fois sur un batch 64x1x32x32

In [ ]:
print("TODO")

**SIGNALEZ que vous avez fini la question ci dessus -- il est nécessaire d'utiliser un codage efficace des permutations pour continuer -- il vous sera fourni**


**Q4 (facile) :**
Regarder les images avant et après permutation -- pourriez vous (vous même) classer les images APRES permutation ??


In [ ]:
print("TODO")

**Q5 (facile)** :
Qu'est ce que ça change pour le MLP ??
-> relancer l'apprentissage/test en ajoutant cette permutation pour valider que ça ne change rien !


In [ ]:
print("TODO")

**Q6 (facile)** : relancer le lenet -> là ça va changer 

In [ ]:
print("TODO")


**Q7 (moyenne) :** faites une fonction qui ajoute du bruit (pour chaque pixel, mettez le à 0 ou à 1 avec une petite probabilité - le bruit est différent pour chaque image train et test)


In [ ]:
print("TODO")

**Q8 (facile)** : Regarder les images avant et après permutation -- pourriez vous (vous même) classer les images APRES permutation ??




In [ ]:
print("TODO")

**Q9 (facile)** : Tester le MLP et le CNN -> cette fois c'est le CNN qui devrait être "plutôt" invariant (le bruit est +/- filtré par la convolution) alors que le MLP devrait être gêné par le bruit !

In [ ]:
print("TODO")

## conclusion

**-> se souvenir que si on prend TOUS les problèmes, tous les algos de classification sont tous aussi mauvais**

**-> ce qui compte c'est qu'ils soient adaptés aux données qu'on peut réellement rencontrer**

**En comparant les images avec permutation des pixels vs avec un bruit... Vous pouvez comprendre pourquoi le CNN est plus adapté à l'image naturelle 
que le MLP !**